In [0]:
%pip install openpyxl

import pandas as pd

"""
Check if this is not the initial run of this notebook i.e.; table named "result" exists
count the number of columns in the table named "result
If the nos of columns is less than nos of columns in existing file at the website
It means a new data (columns) has been added to the existing file at the website.
Append the new data to the existing file.

"""

table_exists = spark.sql("SHOW TABLES LIKE 'result'").count() > 0

if table_exists:


    # Read the new Excel file and load the "Quarter" sheet into a Pandas DataFrame
    df_new = pd.read_excel("https://assets.publishing.service.gov.uk/media/66a76ff1ab418ab055592e8a/ET_3.1_JUL_24.xlsx", sheet_name="Quarter", header=4)

    # Convert the Pandas DataFrame to a Spark DataFrame
    df_new = spark.createDataFrame(df_new)

    # Check for new data by comparing the nos of columns in new Spark DataFrame with the existing one

    previous_columns = len(spark.table("Results").columns)
    new_columns = len(df_new.columns)

    if (new_columns > previous_columns):

        from pyspark.sql.functions import col, monotonically_increasing_id

        # Load the two tables as DataFrames
        df_table2 = spark.table("Results")
        df_table1 = df_new

        # Get the name of the last column in table1
        last_column_name = df_table1.columns[-1]

        # Select the last column from table1
        last_column_df = df_table1.select(last_column_name)

        # Add a monotonically increasing id to both DataFrames to ensure a one-to-one mapping
        df_table2_with_id = df_table2.withColumn("row_id", monotonically_increasing_id())
        last_column_df_with_id = last_column_df.withColumn("row_id", monotonically_increasing_id())

        # Join the last column of table1 to table2 based on the row_id and then drop the row_id

        df_table2_with_last_column = df_table2_with_id.join(last_column_df_with_id, df_table2_with_id["row_id"] == last_column_df_with_id["row_id"], "left").drop("row_id")

         # Write the Spark DataFrame with new column appended to a csv table
         
        df_table2_with_last_column.write.format("csv").mode("overwrite").saveAsTable("Results")


else:

    # It is an initial load, simply write the data to the csv table
    
    # Read the Excel file and load the "Quarter" sheet into a Pandas DataFrame
    df = pd.read_excel("https://assets.publishing.service.gov.uk/media/66a76ff1ab418ab055592e8a/ET_3.1_JUL_24.xlsx", sheet_name="Quarter", header=4)

    # Convert the Pandas DataFrame to a Spark DataFrame
    spark_df = spark.createDataFrame(df)

    # Write the Spark DataFrame to a csv table
    spark_df.write.format("csv").mode("overwrite").saveAsTable("Results")









  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/96/c2/3dd434b0108730014f1b96fd286040dc3bcb70066346f7e01ec2ac95865f/et_xmlfile-1.1.0-py3-none-any.whl.metadata
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
